In [ ]:
#@title Step 1 -- press play if you are on phone{ display-mode: "form" }
%%html
<audio autoplay="" src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" loop controls>

In [ ]:
#@title Step 2 -- press play this to run TavernAI with koboldcpp

Model = "https://huggingface.co/bartowski/magnum-v4-12b-GGUF/resolve/main/magnum-v4-12b-Q5_K_M.gguf"
use_drive_for_chat_backup = True #@param {type:"boolean"}
import os
import subprocess
import time
import shutil
import threading
import torch
import sys
import requests
if not torch.cuda.is_available():
    from IPython.display import display, Markdown
    display(Markdown("<h1>NO free gpu is avilable for you at the moment</h1>"))
    sys.exit()
from google.colab import drive
os.chdir("/content")
print("Chào mừng đến với TavernAI cùng Koboldcpp")
print("Sổ tay Colab được viết bởi doctord98 với sự hỗ trợ api bổ sung của vrihatgan")
print("Vui lòng đợi trong khi tôi thiết lập mọi thứ cho bạn")
if use_drive_for_chat_backup:
  drive.mount('/content/drive')
subprocess.call("git clone https://github.com/Akimitsujiro/TAI.git", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
src1 = '/content/TavernAI/public/characters'
des1 = '/content/drive/MyDrive/TavernAI/characters'
src2 = '/content/TavernAI/public/chats'
des2 = '/content/drive/MyDrive/TavernAI/chats'
Layers = 99
ContextSize = 6144
subprocess.run("wget -O dlfile.tmp https://kcpplinux.concedo.workers.dev && mv dlfile.tmp koboldcpp_linux", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run("chmod +x ./koboldcpp_linux", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
subprocess.run("apt install aria2 -y", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("Tải mô hình")
subprocess.run(f"aria2c -x 10 -o model.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none {Model}", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
print("Tải thành công...")
print("đang tải mô hình...")
command = f"./koboldcpp_linux model.gguf --usecublas 0 mmq --multiuser --gpulayers {Layers} --contextsize {ContextSize} --quiet"
subprocess.Popen(command, shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
flag_file = "backup_flag.txt"
if use_drive_for_chat_backup:
    if os.path.exists(flag_file):
        print("Mã đã được thực thi.")
    else:

      if not os.path.exists(des1):
        os.makedirs(des1)
        shutil.copytree(src1, des1, dirs_exist_ok=True)
      else:
        !rm -rf /content/TavernAI/public/characters/*
        shutil.copytree(des1, src1, dirs_exist_ok=True)
      if not os.path.exists(des2):
        os.makedirs(des2)
        shutil.copytree(src2, des2, dirs_exist_ok=True)
      else:
        !rm -rf /content/TavernAI/public/chats/*
        shutil.copytree(des2, src2, dirs_exist_ok=True)



      def backup_thread():
        while True:
          try:
            shutil.copytree(src1, des1, dirs_exist_ok=True)
            shutil.copytree(src2, des2, dirs_exist_ok=True)
          except Exception as e:
            pass

          time.sleep(5)

      backup_thread = threading.Thread(target=backup_thread)
      backup_thread.start()

      with open(flag_file, "w") as f:
        f.write("bạn không cần phải chạy lại mã này chỉ cần chạy mã bên dưới nếu liên kết không hoạt động")

subprocess.call("npm install -g npm@8.19.3", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
os.chdir("/content/TavernAI")
subprocess.call("npm install", shell=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
os.chdir('/content/')
if not os.path.isfile('cloudflared-linux-amd64'):
    os.system('wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64')
    os.chmod('cloudflared-linux-amd64', 0o777)

os.system('nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:8000 --metrics 127.0.0.1:31337 > cf.txt 2>&1 &')

time.sleep(10)

def is_service_up(url, timeout=5):
    try:
        response = requests.get(url, timeout=timeout)
        return response.status_code == 200
    except requests.ConnectionError:
        return False

metrics_url = 'http://127.0.0.1:31337/metrics'
if is_service_up(metrics_url):
    try:
        scrape = requests.get(metrics_url).text
        needle = scrape.partition('cloudflared_tunnel_user_hostnames_counts{userHostname="')[2].split('"} 1')[0]

        if needle:
            with open('cloudflare.log', 'w') as log:
                log.write("CLOUDFLARE PROVIDES!" + needle)
        print(needle)
    except requests.RequestException as e:
        print(f"Lỗi khi lấy số liệu: {e}")
else:
    print("Dịch vụ số liệu không khả dụng.")

os.chdir("/content/TavernAI")
!nohup node server.js >/dev/null 2>&1
